In [1]:
import pandas as pd
import re

## Load the Data
First, load the trash hauler CSV and preview it.

In [2]:
df = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')
print(f"Total rows: {len(df)}")
df.head()

Total rows: 20226


,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


## Step 1: Basic Text Search
Let's start simple - find rows where the Description contains the word "missed".

**Concept**: `.str.contains()` searches for text patterns in a pandas Series.

In [3]:
# Find all rows containing "missed"
missed = df['Description'].str.contains('missed', case=False, na=False)
print(f"Found {missed.sum()} rows with 'missed'")
df[missed][['Description']].head(10)

Found 10785 rows with 'missed'


,Description
3,missed
4,Missed the even side of the road.
8,Missed.
9,"Trash missed Tuesday again, ALLEY"
10,Missed trash pickup - resident has at curb for...
11,Missed- 4th week in a row.
12,Missed. Does not have access to alley.
14,Missed.
15,Pickup missed
19,Missed.


## Step 2: Word Boundaries
Problem: "missed" also matches "dismissed". Use `\b` for exact word matches.

**Pattern**: `\bmissed\b` means "the word 'missed' with boundaries on both sides"

In [4]:
# Use word boundaries for exact match
missed_exact = df['Description'].str.contains(r'\bmissed\b', case=False, na=False, regex=True)
print(f"Found {missed_exact.sum()} rows with exact word 'missed'")
df[missed_exact][['Description']].head(10)

Found 10781 rows with exact word 'missed'


,Description
3,missed
4,Missed the even side of the road.
8,Missed.
9,"Trash missed Tuesday again, ALLEY"
10,Missed trash pickup - resident has at curb for...
11,Missed- 4th week in a row.
12,Missed. Does not have access to alley.
14,Missed.
15,Pickup missed
19,Missed.


## Step 3: OR Patterns (Alternation)
Find rows mentioning "missed" OR "pickup" OR "alley".

**Pattern**: `missed|pickup|alley` uses `|` to mean OR

In [11]:
# Search for multiple keywords
# regex=True is needed because | (pipe) is a regex operator meaning OR
keywords = df['Description'].str.contains(r'missed|pickup|alley', case=False, na=False, regex=True)
print(f"Found {keywords.sum()} rows with any of these words")
df[keywords][['Description']].head(10)

Found 12414 rows with any of these words


,Description
3,missed
4,Missed the even side of the road.
5,left trash cart in middle of driveway instead ...
7,"Trash out on time, miss again Tuesday. ALLEY"
8,Missed.
9,"Trash missed Tuesday again, ALLEY"
10,Missed trash pickup - resident has at curb for...
11,Missed- 4th week in a row.
12,Missed. Does not have access to alley.
14,Missed.


## Step 4: Digits and Quantifiers
Extract ZIP codes (5 digits).

**Pattern**: `\d{5}` means "exactly 5 digits"
- `\d` = any digit (0-9)
- `{5}` = exactly 5 times

In [6]:
# Extract 5-digit ZIP codes from Description
df['extracted_zip'] = df['Description'].str.extract(r'(\d{5})')
df[df['extracted_zip'].notna()][['Description', 'extracted_zip']].head(10)

,Description,extracted_zip
79,Trash missed last Thursday. Does not understan...,23455
82,not putting carts back where they got them lid...,90480
147,The whole court miss trash/Friday. Also Roundw...,37013
190,My trash not collected but neighbor's trash wa...,37206
202,Trash was not picked up at 244 Ash Grove Dr 37211,37211
276,Trash picked up this morning and driver droppe...,37211
363,2867 Creekbend Drive - 37207 - My trash NOT pi...,37207
372,"Trash was not picked up last Thursday, making ...",37204
404,ref 298388 / cw 939492 is not satisfied with r...,29838
444,None of the Trash was not picked up on Wednesd...,37215


## Step 5: Optional Patterns
ZIP codes can be 5 digits OR 5 digits + 4 more (ZIP+4 format).

**Pattern**: `\d{5}(?:-\d{4})?`
- `(?:...)` = non-capturing group
- `?` = optional (0 or 1 times)

In [7]:
# Extract ZIP codes with optional +4 extension
df['zip_plus4'] = df['Description'].str.extract(r'(\d{5}(?:-\d{4})?)')
df[df['zip_plus4'].notna()][['Description', 'zip_plus4']].head(10)

,Description,zip_plus4
79,Trash missed last Thursday. Does not understan...,23455
82,not putting carts back where they got them lid...,90480
147,The whole court miss trash/Friday. Also Roundw...,37013
190,My trash not collected but neighbor's trash wa...,37206
202,Trash was not picked up at 244 Ash Grove Dr 37211,37211
276,Trash picked up this morning and driver droppe...,37211
363,2867 Creekbend Drive - 37207 - My trash NOT pi...,37207
372,"Trash was not picked up last Thursday, making ...",37204
404,ref 298388 / cw 939492 is not satisfied with r...,29838
444,None of the Trash was not picked up on Wednesd...,37215


## Step 6: Phone Numbers
Extract phone numbers like `615-876-6274` or `(615) 569-4994`.

**Pattern**: `\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}`
- `\(?` = optional opening parenthesis
- `[\s.-]?` = optional space, dot, or dash

In [8]:
# Extract phone numbers
df['phone'] = df['Description'].str.extract(r'(\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})')
df[df['phone'].notna()][['Description', 'phone']].head(10)

,Description,phone
0,"house with the wheel chair ramp, they share dr...",615-876-6274
13,trash truck took cart \r\n3 guys on truck did ...,(615) 569-4994
39,Alley trash miss for Wednesday. Wants cart emp...,615-293-1573
63,Missed trash pickup said has been picking up b...,(615) 333-0065
64,Missed again\r\npicked up everyone except her\...,(615) 753-1696
79,Trash missed last Thursday. Does not understan...,513-403-0759
80,missed pick up - please call (615) 519-3192 wa...,(615) 519-3192
82,not putting carts back where they got them lid...,615-975-1015
139,"Backdoor trash keeps being missed, truck comes...",615-982-6915
171,Curb/Trash miss for pick up Tuesday. Out Monda...,615-227-9888


## Step 7: Street Addresses
Extract addresses like `3817 Crouch Dr` or `4028 Clarksville Pike`.

**Pattern**: `\d{1,5}\s+[A-Za-z\s]+(?:St|Dr|Ave|Rd|Pike|Blvd|Ct|Ln)`
- `\d{1,5}` = 1 to 5 digits (house number)
- `\s+` = one or more spaces
- `[A-Za-z\s]+` = street name (letters and spaces)
- `(?:St|Dr|...)` = street type

In [9]:
# Extract street addresses
address_pattern = r'\d{1,5}\s+[A-Za-z\s]+(?:St|Dr|Ave|Rd|Pike|Blvd|Ct|Ln|Way|Pl)'
df['address_extracted'] = df['Description'].str.extract(f'({address_pattern})')
df[df['address_extracted'].notna()][['Description', 'address_extracted']].head(10)

,Description,address_extracted
0,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr
33,"5th time in as many weeks, Metro has not picke...",3702 Hobbs Rd
97,While cleaning the alley behind 1619 Heiman St...,1619 Heiman St
103,While cleaning the alley behind 1619 Heiman St...,1619 Heiman St
124,Missed pickup in alley behind 4611 Alabama Ave,4611 Alabama Ave
190,My trash not collected but neighbor's trash wa...,264 Riverside Dr
202,Trash was not picked up at 244 Ash Grove Dr 37211,244 Ash Grove Dr
350,3554 Gondola Dr. Missed one side of the road.,3554 Gondola Dr
363,2867 Creekbend Drive - 37207 - My trash NOT pi...,2867 Creekbend Dr
512,Says the trash was missed in the 900 and 800 b...,800 blocks of Lischey Ave


## Step 8: Days of the Week
Find which day pickups were missed.

**Pattern**: `(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)`

In [12]:
# Extract day of week
days_pattern = r'(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday|Mon|Tue|Tues|Wed|Thu|Thurs|Fri|Sat|Sun)'
df['day_mentioned'] = df['Description'].str.extract(days_pattern, flags=re.IGNORECASE)
df[df['day_mentioned'].notna()][['Description', 'day_mentioned']].head(10)

,Description,day_mentioned
1,Curb/Trash miss Tuesday.,Tuesday
2,Curb/trash miss Tuesday.,Tuesday
6,Trash/emptied Wednesday & now metal black-mail...,Wednesday
7,"Trash out on time, miss again Tuesday. ALLEY",Tuesday
9,"Trash missed Tuesday again, ALLEY",Tuesday
18,Curb/trash miss for this address and others. T...,Tuesday
21,"Our trash was not picked up today, Wed, Nov 1....",Wed
22,"Alley trash miss Tuesday, also others as well,...",Tuesday
38,trash missed Wednesday. Curb,Wednesday
39,Alley trash miss for Wednesday. Wants cart emp...,Wednesday


## Step 9: Character Classes
Find truck numbers like `4887AH` (digits followed by letters).

**Pattern**: `\d{4}[A-Z]{2}`
- `[A-Z]` = any uppercase letter
- `{2}` = exactly 2 times

In [13]:
# Extract truck numbers (pattern: 4 digits + 2 letters)
df['truck_number'] = df['Description'].str.extract(r'(\d{4}[A-Z]{2})')
df[df['truck_number'].notna()][['Description', 'truck_number']].head()

,Description,truck_number
20,A resident is complaining that a trash truck r...,4887AH
1805,There was a Public Works trash truck driving d...,4908AG
5941,I need a 105 property damaged filled out pleas...,5219AH
6402,brush truck # 4249AC pulled an at&t line from ...,4249AC
10377,Driver almost caused a wreck. Bumper#5123AH,5123AH


## Step 10: Anchors (Start and End)
Check if a description STARTS with a number.

**Pattern**: `^\d+`
- `^` = start of string
- `$` = end of string
- `+` = one or more times

In [16]:
# Find descriptions starting with a number
# Use .str.contains() with ^ anchor instead of .str.match() to avoid NaN issues
starts_with_number = df['Description'].str.contains(r'^\d+', na=False)
print(f"Descriptions starting with a number: {starts_with_number.sum()}")
df[starts_with_number][['Description']].head()

Descriptions starting with a number: 773


,Description
33,"5th time in as many weeks, Metro has not picke..."
227,"8 houses missed last Thursday, Every other wee..."
346,626 40th Ave N - Clifton Ave Market- 4 trash c...
350,3554 Gondola Dr. Missed one side of the road.
359,"2 trash/carts curb, full carts. Needs to be em..."


## Practice Challenge 1: Count Complaint Types
Tag rows by complaint type keywords.

In [17]:
# Create flags for different complaint types
df['is_missed_pickup'] = df['Request'].str.contains(r'Missed Pickup', case=False, na=False)
df['is_damage'] = df['Request'].str.contains(r'Damage', case=False, na=False)
df['is_backdoor'] = df['Request'].str.contains(r'Backdoor', case=False, na=False)

# Count each type
print(f"Missed Pickups: {df['is_missed_pickup'].sum()}")
print(f"Damage Complaints: {df['is_damage'].sum()}")
print(f"Backdoor Requests: {df['is_backdoor'].sum()}")

Missed Pickups: 15028
Damage Complaints: 257
Backdoor Requests: 2629


## Practice Challenge 2: Extract Request Numbers
The first column has request numbers (e.g., 25270). Extract all unique request numbers from descriptions that mention them.

In [19]:
# Extract 5-digit numbers that might be request references
request_refs = df['Description'].str.findall(r'\b\d{5}\b')
# Count how many descriptions reference other requests
# Handle NaN values by checking if x is a list first
has_reference = request_refs.apply(lambda x: len(x) > 0 if isinstance(x, list) else False)
print(f"Descriptions mentioning 5-digit numbers: {has_reference.sum()}")
df[has_reference][['Request Number', 'Description']].head()

Descriptions mentioning 5-digit numbers: 186


,Request Number,Description
79,25862,Trash missed last Thursday. Does not understan...
147,26391,The whole court miss trash/Friday. Also Roundw...
190,27220,My trash not collected but neighbor's trash wa...
202,27318,Trash was not picked up at 244 Ash Grove Dr 37211
276,28382,Trash picked up this morning and driver droppe...


## Practice Challenge 3: Normalize Text
Clean up descriptions by:
1. Removing extra whitespace
2. Converting to lowercase
3. Removing punctuation at the end

In [ ]:
# Clean descriptions
df['clean_description'] = (df['Description']
    .str.strip()                           # Remove leading/trailing spaces
    .str.replace(r'\s+', ' ', regex=True)  # Collapse multiple spaces
    .str.replace(r'[.!,]+$', '', regex=True)  # Remove trailing punctuation
    .str.lower()                           # Convert to lowercase
)

# Compare original vs cleaned
df[['Description', 'clean_description']].head(10)

## Summary: Regex Quick Reference

| Pattern | Meaning | Example |
|---------|---------|----------|
| `\d` | Any digit | `\d{3}` = 3 digits |
| `\w` | Word character | `\w+` = one or more letters/digits |
| `\s` | Whitespace | `\s+` = one or more spaces |
| `[abc]` | One of a, b, or c | `[0-9]` = any digit |
| `[^abc]` | NOT a, b, or c | `[^0-9]` = not a digit |
| `.` | Any character | `a.c` matches abc, a2c, etc. |
| `*` | 0 or more | `a*` = zero or more a's |
| `+` | 1 or more | `a+` = one or more a's |
| `?` | 0 or 1 (optional) | `colou?r` = color or colour |
| `{n}` | Exactly n times | `\d{5}` = exactly 5 digits |
| `{n,m}` | Between n and m | `\d{3,5}` = 3 to 5 digits |
| `\|` | OR | `cat\|dog` = cat or dog |
| `^` | Start of string | `^The` = starts with "The" |
| `$` | End of string | `end$` = ends with "end" |
| `\b` | Word boundary | `\bcat\b` = the word "cat" |
| `(...)` | Capture group | `(\d+)` captures digits |
| `(?:...)` | Non-capturing group | `(?:Mr\|Mrs)` groups without capturing |

## Your Turn!

Try these exercises:

1. **Find complaints about carts**: Search for descriptions mentioning "cart" or "bin"
2. **Extract times**: Look for patterns like "7 am" or "3:30 pm"
3. **Find repeated words**: Use `(\w+)\s+\1` to find repeated words
4. **Extract council districts**: Pull the district numbers from the data
5. **Find all-caps words**: Use `[A-Z]{2,}` to find words in all caps

Good luck! 🎉